In [1]:
import pandas as pd

In [2]:
data = [[1, 2, '2016/06/03'], [1, 3, '2016/06/08'], [2, 3, '2016/06/08'], [3, 4, '2016/06/09']]
request_accepted = pd.DataFrame(data, columns=['requester_id', 'accepter_id', 'accept_date']).astype({'requester_id':'Int64', 'accepter_id':'Int64', 'accept_date':'datetime64[ns]'})
request_accepted

,requester_id,accepter_id,accept_date
0,1,2,2016-06-03
1,1,3,2016-06-08
2,2,3,2016-06-08
3,3,4,2016-06-09


In [ ]:
req = request_accepted['requester_id']
acc = request_accepted['accepter_id']
req_acc = pd.concat([req, acc])
req_acc.value_counts()
cnt = req_acc.value_counts()
cnt[cnt == cnt.max()]
pd.DataFrame({'id': cnt[cnt == cnt.max()].index, 'num': cnt[cnt == cnt.max()].values})

3    3
1    2
2    2
4    1
Name: count, dtype: Int64

value_counts() 사용하는 방법

In [44]:
request_accepted.groupby('requester_id')['accepter_id'].count()

requester_id
1    2
2    1
3    1
Name: accepter_id, dtype: Int64

In [49]:
rv = request_accepted['requester_id'].value_counts()
av = request_accepted['accepter_id'].value_counts()
ans = rv.add(av, fill_value=0)
print((ans.idxmax()))
print((ans.max()))

3
3


## 607. Sales Person

Write a solution to find the names of all the salespersons who did not have any orders related to the company with the name "RED".

Return the result table in any order.

The result format is in the following example.

In [63]:
data = [[1, 'John', 100000, 6, '4/1/2006'], [2, 'Amy', 12000, 5, '5/1/2010'], [3, 'Mark', 65000, 12, '12/25/2008'], [4, 'Pam', 25000, 25, '1/1/2005'], [5, 'Alex', 5000, 10, '2/3/2007']]
sales_person = pd.DataFrame(data, columns=['sales_id', 'name', 'salary', 'commission_rate', 'hire_date']).astype({'sales_id':'Int64', 'name':'object', 'salary':'Int64', 'commission_rate':'Int64', 'hire_date':'datetime64[ns]'})
data = [[1, 'RED', 'Boston'], [2, 'ORANGE', 'New York'], [3, 'YELLOW', 'Boston'], [4, 'GREEN', 'Austin']]
company = pd.DataFrame(data, columns=['com_id', 'name', 'city']).astype({'com_id':'Int64', 'name':'object', 'city':'object'})
data = [[1, '1/1/2014', 3, 4, 10000], [2, '2/1/2014', 4, 5, 5000], [3, '3/1/2014', 1, 1, 50000], [4, '4/1/2014', 1, 4, 25000]]
orders = pd.DataFrame(data, columns=['order_id', 'order_date', 'com_id', 'sales_id', 'amount']).astype({'order_id':'Int64', 'order_date':'datetime64[ns]', 'com_id':'Int64', 'sales_id':'Int64', 'amount':'Int64'})
display(sales_person)
display(company)
display(orders)

,sales_id,name,salary,commission_rate,hire_date
0,1,John,100000,6,2006-04-01
1,2,Amy,12000,5,2010-05-01
2,3,Mark,65000,12,2008-12-25
3,4,Pam,25000,25,2005-01-01
4,5,Alex,5000,10,2007-02-03


,com_id,name,city
0,1,RED,Boston
1,2,ORANGE,New York
2,3,YELLOW,Boston
3,4,GREEN,Austin


,order_id,order_date,com_id,sales_id,amount
0,1,2014-01-01,3,4,10000
1,2,2014-02-01,4,5,5000
2,3,2014-03-01,1,1,50000
3,4,2014-04-01,1,4,25000


In [81]:
red_id = company[company['name'] == 'RED']['com_id']
red_order_id = orders[orders['com_id'].isin(red_id)]['sales_id']
pd.DataFrame(sales_person[~sales_person['sales_id'].isin(red_order_id)]['name'])

,name
1,Amy
2,Mark
4,Alex


### **merge 연습하기**

In [ ]:
def sales_person(sales_person: pd.DataFrame, company: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    df1 = pd.merge(orders,company,on='com_id', how='left')[['sales_id','name']]
    df=  pd.merge(sales_person,df1, on='sales_id',how='left').rename(columns={'name_x':'name','name_y':'company_name'})[['name','company_name']]
    df = df[df['company_name']=="RED"][['name']]
    df3 = sales_person[~sales_person['name'].isin(df['name'])][['name']]
    return df3

2    1
3    4
Name: sales_id, dtype: Int64

In [85]:
df1 = pd.merge(orders,company,on='com_id', how='left')[['sales_id','name']]
pd.merge(sales_person,df1, on='sales_id',how='left')

,sales_id,name_x,salary,commission_rate,hire_date,name_y
0,1,John,100000,6,2006-04-01,RED
1,2,Amy,12000,5,2010-05-01,NaN
2,3,Mark,65000,12,2008-12-25,NaN
3,4,Pam,25000,25,2005-01-01,YELLOW
4,4,Pam,25000,25,2005-01-01,RED
5,5,Alex,5000,10,2007-02-03,GREEN


## 608. Tree Node

Each node in the tree can be one of three types:

- "Leaf": if the node is a leaf node.
- "Root": if the node is the root of the tree.
- "Inner": If the node is neither a leaf node nor a root node.

Write a solution to report the type of each node in the tree.

Return the result table in any order.

The result format is in the following example.

In [87]:
data = [[1, None], [2, 1], [3, 1], [4, 2], [5, 2]]
tree = pd.DataFrame(data, columns=['id', 'p_id']).astype({'id':'Int64', 'p_id':'Int64'})
display(tree)

,id,p_id
0,1,<NA>
1,2,1
2,3,1
3,4,2
4,5,2


In [126]:
start = tree['id'].unique()
end = tree['p_id'].unique()
leaf = tree[~tree['id'].isin(end)]['id']
root = tree[~tree['id'].isin(tree.dropna()['id'])]['id']
inner = tree[~tree['id'].isin(pd.concat([root, leaf]))]['id']

In [127]:
pd.DataFrame({'id': pd.concat([root, inner, leaf]), 'type': ['Root']*len(root) + ['Inner']*len(inner) + ['Leaf']*len(leaf)})

,id,type
0,1,Root
1,2,Inner
2,3,Leaf
3,4,Leaf
4,5,Leaf


- id 중에 p_id에 없으면 leaft 노드
- p_id null인 id는 root 노드
- 나머지는 inner 노드

In [ ]:
def tree_node(tree: pd.DataFrame) -> pd.DataFrame:
    if tree.empty:
        return pd.DataFrame(columns=['id', 'type'])
    
    parents_node = tree['p_id'].unique()

    node_id = tree['id']
    parent_id = tree['id']
    for row in tree
    if pd.isna(parent_id):
        

False